In [10]:
from __future__ import print_function
import os, sys
import pickle
import os.path
import io
import json

import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as scs

from matplotlib import pyplot as plt
import seaborn as sns


from io import StringIO
from math import floor
from tqdm import tqdm, trange

from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec


from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account


class google_api():
    
    def __init__(self, token_file = 'token.pickle',fauth = None):
        
        self.SCOPES = (
            #with ability to use files/images stored in drive
            'https://www.googleapis.com/auth/drive',
            #to edit slides
            'https://www.googleapis.com/auth/presentations',
            #to sheet
            'https://www.googleapis.com/auth/spreadsheets.readonly'
        )
        
        #get credential file
        self.HOME = os.environ.get('HOME')
        
        if fauth is None:
            fauth = 'bioreport.json'

        if os.path.exists('./.env'):
            path = './.env'
        else:
            path = os.path.join(self.HOME, '.credentials')
            
        self.fauth = os.path.join(path, fauth)            

        self.token_file = token_file

        auth = json.load(open(self.fauth,'r'))
        if auth.get("type",'')=="service_account":
            #print(f'****** using service account: {self.fauth}')
            self.creds = self.get_service_account()
        else:
            self.creds = self.get_token()

    def get_service_account(self):
        return service_account.Credentials.from_service_account_file(self.fauth, scopes=self.SCOPES)
        
    
    def get_service(self,name=['drive','sheet','slide']):
        
        s = {}            
        if 'sheet' in name: 
            s['sheet'] = build('sheets', 'v4', credentials=self.creds)
            
        if 'drive' in name:        
            s['drive'] =  build('drive',  'v3', credentials=self.creds)
            
        if 'slide' in name:
            s['gslide'] = build('slides', 'v1', credentials=self.creds)
        
        return s

    
class gsheet(google_api):
    
    def __init__(self,sheetid, ftoken = 'token.pickle',
                 fauth = None,
                 sheetname = None):        
        #        
        super().__init__(fauth = fauth, token_file = ftoken)
        #print(ftoken, type(ftoken))
        #
        self.sheetid = sheetid
        self.sheetname = sheetname
        
        s = self.get_service()
        self.drive = s['drive'] 
        self.gsheet = s['sheet']    
        
        self.sheet = self.gsheet.spreadsheets()
 

    def get_sheet_df(self,name, start='A',
                     end=None, iheader=0, idata=0, lastrow=0,    
                     major='COLUMNS'):  
        
        #get header
        if iheader>0:
            hrange = f'{name}!{start}{iheader}:{end}{iheader}'
            result = self.sheet.values().get(spreadsheetId=self.sheetid, 
                                                    range=hrange).execute()
            if idata==1:
                idata=2            
            self.all_columns = result.get('values', [])[0]
        else:
            iheader=0
            nrows = lastrow-idata+1
            self.all_columns=['']*nrows
            
        
        
        #get data
        if lastrow==0:
            if end is None:
                drange = f'{name}'
            else:
                drange = f'{name}!{start}:{end}'
        else:
            drange = f'{name}!{start}{idata}:{end}{lastrow}'

        result = self.sheet.values().get(spreadsheetId=self.sheetid, 
                                         range=drange,
                                         majorDimension=major).execute()
         
        values = result['values']

        if lastrow==0:
            df = pd.DataFrame(values).transpose()
            col = df.iloc[0].values
            df = df.drop([0])
            df.columns = col
        else:
            nrows = lastrow-idata+1
            data = {col:['']*nrows for col in self.key_columns}
            for col in self.key_columns:
                if col in self.all_columns:
                    ic = self.all_columns.index(col)
                    v = values[ic]
                    nv = len(v)
                    data[col][0:nv] = v
                        
            df = pd.DataFrame.from_dict(data)
        

        #df = df[:,self.key_columns].dropna(axis='columns', thresh=0.95) 
        
        return df.T

    def sheet_update(self,body, major='COLUMNS'): 
        '''
        if you want to update multiple ranges 
        Refer: https://developers.google.com/sheets/api/guides/values
        '''
        
        result = self.sheet.values().batchUpdate(spreadsheetId=self.sheetid, 
                                                 body=body).execute()

        print('{0} cells updated.'.format(result.get('totalUpdatedCells')))        
        
        return result

    def single_update(self,name, row_list, loc='A1', major='COLUMNS'):  
        '''
        if you want to update a single row 
        Refer: https://developers.google.com/sheets/api/guides/values
        
        row_list must be a list of lists: e,g [[cell_1],[cell_2],..]
        '''        

        try:
            if not isinstance(row_list[0],list):
                values = [[x] for x in row_list]
            else:
                values = row_list
        except:
            print('row_list must be a list of lists: e,g [[cell_1],[cell_2],..]')
            return
            
        body = {'values': values}
        
        #get data
        #Ref: https://developers.google.com/sheets/api/guides/values
        value_input_option = 'RAW'
        #drange = f'{name}!{start}{istart}:{end}{iend}'
        drange = f'{name}!{loc}'
        result = self.sheet.values().update(spreadsheetId=self.sheetid, 
                                            range=drange,
                                            valueInputOption=value_input_option,
                                            body=body).execute()

        print('{0} cells updated.'.format(result.get('updatedCells')))        
        
        return result                   

In [2]:
gs = gsheet(sheetid = '1xzxzhJzAQ_yiTQNwr2zkejLXioUIL1C3s3w5HNu6HH8', 
            fauth = 'coronaresponse_10ac.json')
#    
df = gs.get_sheet_df('Week2')
df

,,Name,Stephany Wanjiru,Lawal Ogunfowora,Adah Kibet,Tijesunimi Olashore,Idowu ilekura,JOHN LOTOME,Evander Eghan,Glory Odeyemi,...,Ayebilla Avoka,Victoria Akintomide,Oluwasegun Ajikobi,Johnson Obeng,Rofiah Adeshina,Victor Anisi,Stacy Wairegi,Emmanuel Patrick,Abubakar Alaro,Evelyne Umuhirs
1,Progress Tracking,Monday,Done,Done,Done,Done,Done,Done,Done,Done,...,Done,Done,Done,Done,Done,Done,Done,Done,Done,Done
2,,Tuesday,None,Delayed,Done,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,,Wednesday,None,Done,OnTrack,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,,Thursday,None,,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,,Friday,None,,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,,Saturday,None,,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,,Sunday,None,,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,,,None,,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Weekly Plan,Monday,None,Write Medium Article,,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
10,None,Tuesday,None,Host git page,"setup github pages, get tweeps, skim the papers.",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [4]:
#df.T

In [12]:
gs = gsheet(sheetid = '1xzxzhJzAQ_yiTQNwr2zkejLXioUIL1C3s3w5HNu6HH8', 
            fauth = 'coronaresponse_10ac.json')

values = [
    [
        'Yabebal From Python'
    ],
]

res = gs.single_update('Week2', body, loc='A47')

1 cells updated.
